# Récupération et traitement des données

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import pynsee as yns
import requests
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from cartiflette.s3 import download_vectorfile_url_all

/home/cathu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from pathlib import Path

In [3]:
Path.cwd()

PosixPath('/home/cathu/Documents/ENSAE/Projet_Catherine_Christelle')

## Les aménagements cyclables en Ile de France

In [4]:
a_velo= gpd.read_file('amenagements-velo-en-ile-de-france.geojson')

In [5]:
# Nombre d'entrées
a_velo.shape[0]

112217

In [6]:
a_velo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [7]:
a_velo['longueur'].describe()

count    112217.000000
mean        126.315959
std         174.519817
min           0.000000
25%          24.000000
50%          70.000000
75%         161.000000
max        3984.000000
Name: longueur, dtype: float64

In [8]:
a_velo.sample(n=1)

,osm_id,nom_com,sens_voit,ag,panneaux,moyenn_ech,revetement,highway,insee_com,nom_voie,longueur,petite_ech,nv,ad,geometry
20082,1.123832e+09,Cergy,NC,voie verte uni,FR:C115,11,asphalt,path,95127,None,214,1,hors voirie,voie verte uni,"LINESTRING (2.01301 49.05301, 2.01272 49.05281..."


Pour le moment, nous travaillons sans les données de géolocalisation qui ne sont pas nécessaires aux calculs. Nous créons pour cela une nouvelle dataframe

In [9]:
col_to_keep = ["nom_com", "sens_voit", "ag", "panneaux", "revetement", "highway", "insee_com", "longueur", "nv", "ad"] 

In [10]:
# df_a_velo = a_velo[col_to_keep].copy()

df_a_velo = a_velo

print(df_a_velo.head())

         osm_id            nom_com sens_voit                              ag  \
0  4.014620e+08            Chelles    DOUBLE                            None   
1  4.048666e+08   La Queue-en-Brie        NC  chemin service site propre uni   
2  1.154304e+09            Lésigny        NC                  voie verte uni   
3  3.300503e+08  Pontault-Combault    UNIQUE                       DSC bande   
4  1.104033e+09   Champs-sur-Marne    UNIQUE                            None   

  panneaux moyenn_ech revetement      highway insee_com  \
0     None         32       None      service     77108   
1     None         11       None        track     94060   
2     None         11    asphalt         path     77249   
3     None         22       None  residential     77373   
4     None         22    asphalt  residential     77083   

                     nom_voie  longueur petite_ech           nv  \
0  Rue de la Mare Longue Noue        24          3    limite 30   
1             Allée Jacquette 

### Toutes les pistes ne se valent pas : création de variables d'étude pour les aménagements cyclables

Nous souhaitons étudier la répartition géograhique des aménagements cyclables et particulièrement leur densité en mètre par habitants. Toutefois, l'ensemble des pistes cyclables n'est pas de la même "qualité" : séparée de la route ou non, sens inverse de la circulation, type de revêtement, etc. Pour prendre en compte la qualité des aménagements cyclables, nous pouvons donner des coefficients aux mètres de pistes selon ces différents critères lors du calcul du nombre total de mètres aménagés par commune. Nous allons proposer plusieurs méthodes de calcul selon ces critères.

In [11]:
print(df_a_velo["sens_voit"].unique())
print(df_a_velo["ag"].unique())
print(df_a_velo["revetement"].unique())
print(df_a_velo["highway"].unique())
print(df_a_velo["nv"].unique())
print(df_a_velo["ad"].unique())

['DOUBLE' 'NC' 'UNIQUE']
[None 'chemin service site propre uni' 'voie verte uni' 'DSC bande'
 'autre chemin velo uni' 'piste uni' 'DSC' 'piste trottoir uni'
 'bande uni' 'cheminement trottoir uni' 'goulotte' 'chemin dedie uni'
 'chaucidou' 'cheminement uni' 'DSC piste' 'voie bus uni' 'piste bi'
 'bande bi' 'shoulder uni']
[None 'asphalt' 'compacted' 'concrete' 'unpaved' 'wood' 'paving_stones'
 'ground' 'sett' 'gravel' 'sand' 'paved' 'fine_gravel' 'dirt'
 'cobblestone' 'concrete:plates' 'concrete:lanes' 'grass'
 'cobblestone:flattened' 'metal' 'pebblestone' 'unhewn_cobblestone'
 'bricks' 'tartan' 'pavés' 'vegecol' 'artificial_turf' 'grass_paver'
 'earth' 'bitume' 'à_définir' 'rock']
['service' 'track' 'path' 'residential' 'footway' 'cycleway' 'secondary'
 'tertiary' 'living_street' 'unclassified' 'pedestrian' 'steps' 'primary'
 'primary_link' 'secondary_link' 'tertiary_link' 'trunk_link'
 'motorway_link']
['limite 30' 'hors voirie' None 'z20' 'rue pietonne' 'z30' 'escalier velo'
 'velor

In [12]:
df_a_velo['sens_voit'].isna().sum()

0

In [13]:
df_a_velo['ag'].isna().sum()

68312

In [14]:
df_a_velo['revetement'].isna().sum()

38711

In [15]:
df_a_velo['highway'].isna().sum()

0

In [16]:
df_a_velo['nv'].isna().sum()

9548

In [17]:
df_a_velo['ad'].isna().sum()

68522

In [18]:
print(df_a_velo['highway'].value_counts().get('unclassified', 0))

4025


In [19]:
df_a_velo['adg'] = df_a_velo.apply(lambda row: row['ad'] if pd.notna(row['ad']) else (row['ag'] if pd.notna(row['ag']) else 'unclassified'), axis=1)

In [20]:
print(df_a_velo['adg'].value_counts().get('unclassified', 0))

55496


Seul le critère suivant ne présente pas trop de valeurs manquantes (nommées 'unclassified' dans la base) :  type de route ("highway", cf [documentation OpenstreeMap](https://wiki.openstreetmap.org/wiki/Key:highway) ). Les autres variables ont soit trop de valeurs manquantes soit manquent d'intérêt seule (sens des voitures). Cependant, "highway" est une colonne générale de catégorisation pour les données d'OpenStreetMap et manque de spécificité pour l'étude des pistes cyclables. Nous allons donc proposer deux méthodes de pondération (en plus d'une variable non pondérée) : une avec "highway", une avec "adg" (['nature de la voie'](https://opendata.stif.info/api/datasets/1.0/amenagements-velo-en-ile-de-france/attachments/metadonnees_amenagements_velo_en_ile_de_france_pdf/)), en proposant de normer les chemins dont nous n'avons pas connaissance de la qualité. Le nombre de valeurs manquantes est cependant très important, malgré notre tentative de réduire son ampleur en sommant ad et ag (ce sont les types de voies à gauche et à droite). Roulant à droite en, nous avons donner la priorité à voie à droite.

Nous allons produire 3 variables : deux pondérées censées prendre en compte la qualité de la route, et une autre sans pondération, pour la longueur d'aménagements cyclables par ville. Pour prendre en compte la qualité nous trions les types de routes en leur affectant des poids selon leurs caractéristiques : bande le long d'une route ou séparation, chemin avec ou sans voiture, avec ou sans piéton, etc. Pour cela, la variable "adg" nous semble plus adaptée car plus précise, mais elle a le défaut de présenter énormément valeurs manquantes. Lorsque la valeur est manquante nous assignons un poids neutre (1). La comparaison des résultats avec les 2 méthodes de pondération ou la méthode sans pondération pourra aussi nous renseigner sur la qualité de notre catégorisation.

In [21]:
print(df_a_velo["adg"].unique())

['unclassified' 'chemin service site propre uni' 'voie verte uni'
 'DSC bande' 'bande uni' 'autre chemin velo uni' 'piste uni' 'DSC'
 'piste trottoir uni' 'cheminement trottoir uni' 'goulotte'
 'chemin dedie uni' 'chaucidou' 'cheminement uni' 'voie bus uni'
 'DSC piste' 'piste bi' 'bande bi' 'shoulder uni']


In [22]:
highway_quality_mapping = {
    'service': 1,
    'track': 1,
    'path': 1,
    'trunk_link': 1,
    'motorway_link': 1,
    'residential': 2,
    'footway': 2,
    'cycleway': 2,
    'primary': 2,    
    'primary_link': 2,
    'unclassified': 2,
    'secondary': 3,
    'tertiary': 3,
    'secondary_link': 3,
    'tertiary_link': 3,
    'living_street': 3,
    'pedestrian': 4,
    'steps': 4
}


adg_quality_mapping = {
    'chemin service site propre uni': 1,
    'chemin dedie uni': 1,
    'voie verte uni': 1,
    'autre chemin velo uni': 1,
    'piste bi' : 1,
    'piste uni': 1,
    'bande uni': 2,
    'bande bi' : 2,
    'unclassified': 2,
    'cheminement trottoir uni' : 3,
    'piste trottoir uni': 3,
    'chaucidou': 3,
    'cheminement uni': 3,
    'shoulder uni' : 3,
    'voie bus uni': 4,
    'goulotte': 4,
    'DSC': 4,
    'DSC bande' : 4,
    'DSC piste' : 4
}

In [23]:
df_a_velo['qual_hw'] = df_a_velo['highway'].map(highway_quality_mapping)

df_a_velo['qual_adg'] = df_a_velo['adg'].map(adg_quality_mapping)

In [24]:
quality_weights = {
    1: 1.25,
    2: 1.,
    3: 0.75,
    4: 0.5
}

In [25]:
# Nouvelle colonne pour la longueur pondérée highway

df_a_velo['longueur_pond_hw'] = df_a_velo['longueur'] * df_a_velo['qual_hw'].map(quality_weights)

# Nouvelle colonne pour la longueur pondérée nature voie

df_a_velo['longueur_pond_adg'] = df_a_velo['longueur'] * df_a_velo['qual_adg'].map(quality_weights)

In [26]:
total_longueur_pond_hw = df_a_velo['longueur_pond_hw'].sum()
print(total_longueur_pond_hw) #longueur totale de l'échantillon avec majoration pondérée

14028145.5


In [27]:
total_longueur_pond_nv = df_a_velo['longueur_pond_adg'].sum()
print(total_longueur_pond_nv) #longueur totale de l'échantillon avec majoration pondérée

14382204.5


In [28]:
total_longueur = df_a_velo['longueur'].sum()
print(total_longueur)

14174798


In [29]:
total_longueur_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur'].sum().reset_index()

In [30]:
total_longueur_pond_hw_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond_hw'].sum().reset_index()

In [31]:
total_longueur_pond_adg_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond_adg'].sum().reset_index()

In [32]:
df_amenagements = pd.merge(total_longueur_commune, total_longueur_pond_hw_commune, on=['nom_com', 'insee_com'], suffixes=('_non_pond', '_pond_hw'))

In [33]:
df_amenagements = pd.merge(df_amenagements, total_longueur_pond_adg_commune, on=['nom_com', 'insee_com'], suffixes=(None, '_pond_adg'))

In [34]:
df_amenagements.sample(10)

,nom_com,insee_com,longueur,longueur_pond_hw,longueur_pond_adg
469,Le Port-Marly,78502,4390,4568.50,5094.25
58,Auvernaux,91037,1666,1852.75,1897.75
449,Le Châtelet-en-Brie,77100,7303,7459.75,7516.50
882,Tremblay-en-France,93073,35228,36772.00,39370.50
598,Montigny-le-Bretonneux,78423,81048,80518.25,91675.50
170,Chalou-Moulineux,91131,3619,3530.25,3926.00
718,Puiselet-le-Marais,91508,422,316.50,422.00
873,Tigeaux,77466,30,30.00,37.50
715,Prunay-en-Yvelines,78506,275,343.75,310.25
116,Bouafle,78090,14898,14709.75,15163.50


In [35]:
df_amenagements['difference_hw'] = df_amenagements['longueur_pond_hw'] - df_amenagements['longueur']
df_amenagements['difference_adg'] = df_amenagements['longueur_pond_adg'] - df_amenagements['longueur']

In [114]:
df_amenagements.head(10)

,nom_com,insee_com,longueur,longueur_pond_hw,longueur_pond_adg,difference_hw,difference_adg
0,10e Arrondissement,75110,60281,55764.25,51170.00,-4516.75,-9111.00
1,11e Arrondissement,75111,90116,84508.75,73843.00,-5607.25,-16273.00
2,12e Arrondissement,75112,201748,201066.75,202496.25,-681.25,748.25
3,13e Arrondissement,75113,137731,128770.50,119280.50,-8960.50,-18450.50
4,14e Arrondissement,75114,102618,94183.00,87486.75,-8435.00,-15131.25
5,15e Arrondissement,75115,148909,136007.00,128724.50,-12902.00,-20184.50
6,16e Arrondissement,75116,228240,226273.25,231651.00,-1966.75,3411.00
7,17e Arrondissement,75117,114273,108548.00,100289.50,-5725.00,-13983.50
8,18e Arrondissement,75118,118171,113091.00,98709.50,-5080.00,-19461.50
9,19e Arrondissement,75119,133478,125675.25,128372.00,-7802.75,-5106.00


Nous avons créé nos trois indicateurs par commune : longueur et longueur pondérée (adg et hw) par commune, mais nous avons perdu de l'information dans cette opération puisque nous perdons de ce fait les coordonnées exactes des aménagements. Cependant, pour l'étude de la densité, cela pourra aussi s'avérer utile. Surtout nous avons besoin de ces indicateurs pour le travail économétrique que nous souhaitons mener, où nous considérons alors les communes comme des individus, et la longueur des pistes cyclables comme nous variable d'intérêt. Cependant, avant de passer à la modélisation économétrique pour tenter d'expliquer le développement des aménagements dans les différentes communes, nous souhaitons représenter et analyser spatialement la répartition des aménagements, leur densité (par habitant), leur densité (par kilomètre) et notamment compléter ces analyses grâce à la disponibilité des données du Vélib.

## Données et ressources nécessaires pour l'étude spatiale

In [37]:
communes = download_vectorfile_url_all(
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    values = ["75","77","78","91", "92", "93", "94","95"],
    vectorfile_format="topojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=75/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 40.1kiB [00:00, 2.09MiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=77/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 1.57MiB [00:00, 3.16MiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=78/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


...

Downloading: : 120kiB [00:00, 1.88MiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=94/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 111kiB [00:00, 2.03MiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=95/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 372kiB [00:00, 2.35MiB/s]


Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [38]:
communes['surface'] = communes.to_crs(2154).area

In [39]:
communes.sort_values('surface', ascending = False)

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,INSEE_COG,geometry,surface
111,COMMUNE_0000000009739723,None,Fontainebleau,FONTAINEBLEAU,77186,Sous-préfecture,15696,07,4,77,11,200072346,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77186,"POLYGON ((2.74740 48.48329, 2.74783 48.48327, ...",1.722864e+08
148,COMMUNE_0000002200276627,None,Saint-Germain-en-Laye,SAINT-GERMAIN-EN-LAYE,78551,Sous-préfecture,44806,16,3,78,11,200058519,IGN:EXPRESS-COG-CARTO-TERRITOIRE,78551,"POLYGON ((2.07077 48.94649, 2.07138 48.94676, ...",5.313401e+07
135,COMMUNE_0000002200276622,None,Chenoise-Cucharmoy,CHENOISE-CUCHARMOY,77109,Commune simple,1640,18,3,77,11,200037133,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77109,"POLYGON ((3.13963 48.59977, 3.13951 48.59983, ...",4.874811e+07
229,COMMUNE_0000000009738401,None,Sonchamp,SONCHAMP,78601,Commune simple,1600,14,2,78,11,200073344,IGN:EXPRESS-COG-CARTO-TERRITOIRE,78601,"POLYGON ((1.83232 48.56349, 1.83260 48.56506, ...",4.634931e+07
152,COMMUNE_0000000009739738,None,Étampes,ETAMPES,91223,Sous-préfecture,25629,08,1,91,11,200017846,IGN:EXPRESS-COG-CARTO-TERRITOIRE,91223,"POLYGON ((2.23350 48.39672, 2.23293 48.39656, ...",4.596232e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,COMMUNE_0000000009736527,None,Saint-Mandé,SAINT-MANDE,94067,Commune simple,22377,23,2,94,11,200054781/200057941,IGN:EXPRESS-COG-CARTO-TERRITOIRE,94067,"POLYGON ((2.41900 48.84939, 2.41900 48.84935, ...",9.057590e+05
155,COMMUNE_0000000009734016,None,Gouzangrez,GOUZANGREZ,95282,Commune simple,160,16,3,95,11,200035970,IGN:EXPRESS-COG-CARTO-TERRITOIRE,95282,"POLYGON ((1.90588 49.11131, 1.90559 49.11143, ...",7.725902e+05
181,COMMUNE_0000000009735023,None,Margency,MARGENCY,95369,Commune simple,2846,15,2,95,11,200056380,IGN:EXPRESS-COG-CARTO-TERRITOIRE,95369,"POLYGON ((2.29014 48.99680, 2.28979 48.99688, ...",7.250545e+05
11,COMMUNE_0000000009736027,None,Le Pré-Saint-Gervais,LE PRE-SAINT-GERVAIS,93061,Commune simple,17351,15,1,93,11,200054781/200057875,IGN:EXPRESS-COG-CARTO-TERRITOIRE,93061,"POLYGON ((2.41151 48.88322, 2.41148 48.88315, ...",7.027441e+05


In [40]:
communes.rename(columns={'INSEE_COG': 'insee_com'}, inplace=True)

In [91]:
communes.head(n=10)

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,insee_com,geometry,surface
0,ARR_MUNI0000000009736045,None,Paris 3e Arrondissement,PARIS 3E ARRONDISSEMENT,75103,Capitale d'état,34025,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75103,"POLYGON ((2.35016 48.86199, 2.35019 48.86203, ...",1.170282e+06
1,ARR_MUNI0000000009736046,None,Paris 2e Arrondissement,PARIS 2E ARRONDISSEMENT,75102,Capitale d'état,21595,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75102,"POLYGON ((2.34792 48.87069, 2.34827 48.87062, ...",9.923087e+05
2,ARR_MUNI0000000009736545,None,Paris 4e Arrondissement,PARIS 4E ARRONDISSEMENT,75104,Capitale d'état,29131,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75104,"POLYGON ((2.36849 48.85580, 2.36873 48.85482, ...",1.596122e+06
3,ARR_MUNI0000000009736544,None,Paris 5e Arrondissement,PARIS 5E ARRONDISSEMENT,75105,Capitale d'état,58227,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75105,"POLYGON ((2.33666 48.83967, 2.33672 48.84011, ...",2.543140e+06
4,ARR_MUNI0000000009736543,None,Paris 6e Arrondissement,PARIS 6E ARRONDISSEMENT,75106,Capitale d'état,40303,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75106,"POLYGON ((2.33292 48.85934, 2.33339 48.85924, ...",2.152364e+06
5,ARR_MUNI0000000009736043,None,Paris 9e Arrondissement,PARIS 9E ARRONDISSEMENT,75109,Capitale d'état,60026,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75109,"POLYGON ((2.32589 48.86958, 2.32597 48.87009, ...",2.179290e+06
6,ARR_MUNI0000000009736042,None,Paris 10e Arrondissement,PARIS 10E ARRONDISSEMENT,75110,Capitale d'état,86472,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75110,"POLYGON ((2.36471 48.88437, 2.36527 48.88433, ...",2.890030e+06
7,ARR_MUNI0000000009736035,None,Paris 11e Arrondissement,PARIS 11E ARRONDISSEMENT,75111,Capitale d'état,145208,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75111,"POLYGON ((2.37690 48.87205, 2.37701 48.87195, ...",3.650811e+06
8,ARR_MUNI0000000009736531,None,Paris 13e Arrondissement,PARIS 13E ARRONDISSEMENT,75113,Capitale d'état,180005,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75113,"POLYGON ((2.34390 48.81577, 2.34394 48.81601, ...",7.138222e+06
9,ARR_MUNI0000000009736044,None,Paris 8e Arrondissement,PARIS 8E ARRONDISSEMENT,75108,Capitale d'état,35655,NR,1,75,11,200054781,IGN:EXPRESS-COG-CARTO-TERRITOIRE,75108,"POLYGON ((2.32717 48.88346, 2.32716 48.88323, ...",3.880853e+06


In [90]:
communes['INSEE_COM'] = communes['insee_com']

## Les vélibs en Ile de France

In [42]:
url2 = "https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

In [43]:
velib = gpd.read_file(url2)

In [44]:
velib.sample(n=15)

,capacity,name,stationcode,geometry
319,30,Cité des Sciences et de l'Industrie,19009,POINT (2.38460 48.89660)
685,39,Paul Vaillant Couturier - Chaptal,23001,POINT (2.27753 48.89348)
20,26,Mahatma Gandhi,16123,POINT (2.26421 48.87636)
1247,23,Halle Freyssinet - Parvis Alan Turing,13051,POINT (2.37114 48.83285)
184,30,Charles Gide - Fort de Bicêtre,42701,POINT (2.35398 48.80719)
1422,17,Maine - Antoine Bourdelle,15002,POINT (2.32026 48.84309)
632,30,Louis Ganne - Davout,20045,POINT (2.40973 48.86339)
1163,36,Ecoles - Carmes,5007,POINT (2.34735 48.84890)
1182,34,Paul Doumer - La Tour,16016,POINT (2.28084 48.86012)
304,23,Guy Môquet - Etienne Dolet,22406,POINT (2.29808 48.81527)


In [60]:
print(velib.shape)

(1471, 4)


In [138]:
gdf_velib = velib

In [139]:
gdf_velib.sample(10)

,capacity,name,stationcode,geometry
539,59,Square Cambronne,15010,POINT (2.30257 48.84757)
684,26,Beffroy,22003,POINT (2.26139 48.88659)
781,18,Gabriel Péri - Francisco Ferrer,33102,POINT (2.40734 48.88634)
1298,50,Lac Daumesnil - Saint-Maurice,12120,POINT (2.41837 48.82775)
119,22,Vincent Auriol - Louise Weiss,13049,POINT (2.37039 48.83400)
916,29,Vieille du Temple - Francs Bourgeois,4013,POINT (2.35822 48.85826)
1348,43,Jean Jaurès - Salvador Allende,22402,POINT (2.30266 48.82148)
436,27,Lucien Lanternier - Le Luth,22303,POINT (2.28898 48.93159)
1469,39,Belfort - Voltaire,11020,POINT (2.38299 48.85634)
1044,29,Anatole France - President Roosevelt,33017,POINT (2.38141 48.91552)


On associe désormais aux stations vélib leur localisation (commune) pour créer deux nouvelles variables d'intérêt : la capacité vélib par commune (cap_com) et le nombre de stations vélib par communes (stat_com)

In [140]:
# Étape 1 : Spatial join pour associer à chaque station Vélib le code commune et le nom de la commune
gdf_velib_with_communes = gpd.sjoin(gdf_velib, communes[['id', 'NOM', 'INSEE_COM', 'geometry']], predicate='within')

print(gdf_velib_with_communes.shape)

(1472, 8)


Nous remplaçons un problème avec le code Insee des arrondissements parisiens qu'il faut corriger avant de calculer les indicatrices par arrondissement.

In [141]:
# Créer un dictionnaire de mapping des codes INSEE_COMMUNE pour les arrondissements de Paris
mapping_arrondissements = {
    'Paris 1er Arrondissement': '75101',
    'Paris 2e Arrondissement': '75102',
    'Paris 3e Arrondissement': '75103',
    'Paris 4e Arrondissement': '75104',
    'Paris 5e Arrondissement': '75105',
    'Paris 6e Arrondissement': '75106',
    'Paris 7e Arrondissement': '75107',
    'Paris 8e Arrondissement': '75108',
    'Paris 9e Arrondissement': '75109',
    'Paris 10e Arrondissement': '75110',
    'Paris 11e Arrondissement': '75111',
    'Paris 12e Arrondissement': '75112',
    'Paris 13e Arrondissement': '75113',
    'Paris 14e Arrondissement': '75114',
    'Paris 15e Arrondissement': '75115',
    'Paris 16e Arrondissement': '75116',
    'Paris 17e Arrondissement': '75117',
    'Paris 18e Arrondissement': '75118',
    'Paris 19e Arrondissement': '75119',
    'Paris 20e Arrondissement': '75120'
}

# Appliquer le mapping pour remplacer les codes INSEE_COMMUNE
gdf_velib_with_communes['INSEE_COM'] = gdf_velib_with_communes.apply(lambda row: mapping_arrondissements.get(row['NOM'], row['INSEE_COM']), axis=1)

# Afficher le DataFrame avec les codes INSEE_COMMUNE mis à jour
print(gdf_velib_with_communes)

      capacity                               name stationcode  \
0           21                 Toudouze - Clauzel        9020   
2           14  Abbeville - Faubourg Poissonnière        9002   
33          17                   Choron - Martyrs        9016   
39          22        Godot de Mauroy - Madeleine        9034   
78          24        Victoire - Chaussée d'Antin        9116   
...        ...                                ...         ...   
1159        31       Louis Pasteur - Albert Petit       22206   
1262        20               Stalingrad - Barbara       22208   
1347        28                 Tertres - Fontaine       22203   
1230        27                 Place Aimé Césaire       21402   
1309        33                  Mairie de Clamart       21401   

                      geometry  index_right                        id  \
0     POINT (2.33736 48.87930)            5  ARR_MUNI0000000009736043   
2     POINT (2.34915 48.87922)            5  ARR_MUNI0000000009736043   


In [142]:
# Étape 2 : Créer les variables cap_com et stat_com
df_aggregated = gdf_velib_with_communes.groupby('INSEE_COM').agg(
    cap_com=('capacity', 'sum'),
    stat_com=('index_right', 'count')
).reset_index()

print(df_aggregated.shape)

(82, 3)


In [143]:
df_aggregated.sample(10)

,INSEE_COM,cap_com,stat_com
69,94033,327,11
17,75118,1798,61
35,92044,309,10
0,75101,710,26
5,75106,1028,34
74,94046,180,7
60,93064,103,4
57,93055,563,21
36,92046,238,8
11,75112,2656,67


In [188]:
# Étape 3 : Merge avec le GeoDataFrame des communes
df_velib = pd.merge(df_aggregated, communes, how='left', on='INSEE_COM')

In [189]:
print(df_velib.shape)

(82, 18)


In [190]:
# Liste des colonnes à supprimer
colonnes_a_supprimer = ['ID', 'id','INSEE_CAN', 'INSEE_ARR', 'SIREN_EPCI', 'source', 'insee_com']

# Supprimer les colonnes
df_velib = df_velib.drop(colonnes_a_supprimer, axis=1)

In [191]:
df_velib.head(10)

,INSEE_COM,cap_com,stat_com,NOM,NOM_M,STATUT,POPULATION,INSEE_DEP,INSEE_REG,geometry,surface
0,75101,710,26,Paris 1er Arrondissement,PARIS 1ER ARRONDISSEMENT,Capitale d'état,15917,75,11,"POLYGON ((2.34464 48.85409, 2.34411 48.85428, ...",1.828408e+06
1,75102,747,25,Paris 2e Arrondissement,PARIS 2E ARRONDISSEMENT,Capitale d'état,21595,75,11,"POLYGON ((2.34792 48.87069, 2.34827 48.87062, ...",9.923087e+05
2,75103,437,15,Paris 3e Arrondissement,PARIS 3E ARRONDISSEMENT,Capitale d'état,34025,75,11,"POLYGON ((2.35016 48.86199, 2.35019 48.86203, ...",1.170282e+06
3,75104,712,26,Paris 4e Arrondissement,PARIS 4E ARRONDISSEMENT,Capitale d'état,29131,75,11,"POLYGON ((2.36849 48.85580, 2.36873 48.85482, ...",1.596122e+06
4,75105,1169,36,Paris 5e Arrondissement,PARIS 5E ARRONDISSEMENT,Capitale d'état,58227,75,11,"POLYGON ((2.33666 48.83967, 2.33672 48.84011, ...",2.543140e+06
5,75106,1028,34,Paris 6e Arrondissement,PARIS 6E ARRONDISSEMENT,Capitale d'état,40303,75,11,"POLYGON ((2.33292 48.85934, 2.33339 48.85924, ...",2.152364e+06
6,75107,1060,27,Paris 7e Arrondissement,PARIS 7E ARRONDISSEMENT,Capitale d'état,48888,75,11,"POLYGON ((2.31671 48.84686, 2.31638 48.84677, ...",4.087721e+06
7,75108,1560,51,Paris 8e Arrondissement,PARIS 8E ARRONDISSEMENT,Capitale d'état,35655,75,11,"POLYGON ((2.32717 48.88346, 2.32716 48.88323, ...",3.880853e+06
8,75109,1118,45,Paris 9e Arrondissement,PARIS 9E ARRONDISSEMENT,Capitale d'état,60026,75,11,"POLYGON ((2.32589 48.86958, 2.32597 48.87009, ...",2.179290e+06
9,75110,1619,55,Paris 10e Arrondissement,PARIS 10E ARRONDISSEMENT,Capitale d'état,86472,75,11,"POLYGON ((2.36471 48.88437, 2.36527 48.88433, ...",2.890030e+06


In [192]:
# Fichier intermédiaire :

gdf_velib = gpd.GeoDataFrame(df_velib, geometry='geometry')

gdf_velib.to_file('df_velib.geojson', driver='GeoJSON')


In [193]:
# df_velib_charge = gpd.read_file('df_velib.geojson')

# Afficher les premières lignes du GeoDataFrame chargé
# print(df_velib_charge.head())

## Concaténation des bases d'aménagements cyclables

In [194]:
print(df_velib.columns)

Index(['INSEE_COM', 'cap_com', 'stat_com', 'NOM', 'NOM_M', 'STATUT',
       'POPULATION', 'INSEE_DEP', 'INSEE_REG', 'geometry', 'surface'],
      dtype='object')


In [195]:
print(df_amenagements.columns)

Index(['nom_com', 'insee_com', 'longueur', 'longueur_pond_hw',
       'longueur_pond_adg', 'difference_hw', 'difference_adg'],
      dtype='object')


In [196]:
# Fusion des DataFrames
df_velo = pd.merge(df_amenagements, df_velib, left_on=['insee_com'], right_on =['INSEE_COM'], how='left')

In [197]:
print(df_velo.shape)

(1005, 18)


In [198]:
df_velo.head(15)

,nom_com,insee_com,longueur,longueur_pond_hw,longueur_pond_adg,difference_hw,difference_adg,INSEE_COM,cap_com,stat_com,NOM,NOM_M,STATUT,POPULATION,INSEE_DEP,INSEE_REG,geometry,surface
0,10e Arrondissement,75110,60281,55764.25,51170.00,-4516.75,-9111.00,75110,1619.0,55.0,Paris 10e Arrondissement,PARIS 10E ARRONDISSEMENT,Capitale d'état,86472.0,75,11,"POLYGON ((2.36471 48.88437, 2.36527 48.88433, ...",2.890030e+06
1,11e Arrondissement,75111,90116,84508.75,73843.00,-5607.25,-16273.00,75111,2126.0,57.0,Paris 11e Arrondissement,PARIS 11E ARRONDISSEMENT,Capitale d'état,145208.0,75,11,"POLYGON ((2.37690 48.87205, 2.37701 48.87195, ...",3.650811e+06
2,12e Arrondissement,75112,201748,201066.75,202496.25,-681.25,748.25,75112,2656.0,67.0,Paris 12e Arrondissement,PARIS 12E ARRONDISSEMENT,Capitale d'état,139297.0,75,11,"POLYGON ((2.39007 48.82569, 2.38909 48.82635, ...",1.638084e+07
3,13e Arrondissement,75113,137731,128770.50,119280.50,-8960.50,-18450.50,75113,2256.0,68.0,Paris 13e Arrondissement,PARIS 13E ARRONDISSEMENT,Capitale d'état,180005.0,75,11,"POLYGON ((2.34390 48.81577, 2.34394 48.81601, ...",7.138222e+06
4,14e Arrondissement,75114,102618,94183.00,87486.75,-8435.00,-15131.25,75114,1830.0,55.0,Paris 14e Arrondissement,PARIS 14E ARRONDISSEMENT,Capitale d'état,135592.0,75,11,"POLYGON ((2.34209 48.83834, 2.34208 48.83830, ...",5.613081e+06
5,15e Arrondissement,75115,148909,136007.00,128724.50,-12902.00,-20184.50,75115,3345.0,90.0,Paris 15e Arrondissement,PARIS 15E ARRONDISSEMENT,Capitale d'état,230981.0,75,11,"POLYGON ((2.26279 48.83392, 2.26356 48.83464, ...",8.472060e+06
6,16e Arrondissement,75116,228240,226273.25,231651.00,-1966.75,3411.00,75116,2129.0,65.0,Paris 16e Arrondissement,PARIS 16E ARRONDISSEMENT,Capitale d'état,165523.0,75,11,"POLYGON ((2.27995 48.87857, 2.28010 48.87852, ...",1.640971e+07
7,17e Arrondissement,75117,114273,108548.00,100289.50,-5725.00,-13983.50,75117,2085.0,63.0,Paris 17e Arrondissement,PARIS 17E ARRONDISSEMENT,Capitale d'état,166543.0,75,11,"POLYGON ((2.32742 48.88351, 2.32734 48.88349, ...",5.665304e+06
8,18e Arrondissement,75118,118171,113091.00,98709.50,-5080.00,-19461.50,75118,1798.0,61.0,Paris 18e Arrondissement,PARIS 18E ARRONDISSEMENT,Capitale d'état,192468.0,75,11,"POLYGON ((2.37030 48.90185, 2.37025 48.90177, ...",5.995586e+06
9,19e Arrondissement,75119,133478,125675.25,128372.00,-7802.75,-5106.00,75119,1762.0,61.0,Paris 19e Arrondissement,PARIS 19E ARRONDISSEMENT,Capitale d'état,184573.0,75,11,"POLYGON ((2.37690 48.87205, 2.37537 48.87343, ...",6.784861e+06


In [199]:
print(df_velo.columns)

Index(['nom_com', 'insee_com', 'longueur', 'longueur_pond_hw',
       'longueur_pond_adg', 'difference_hw', 'difference_adg', 'INSEE_COM',
       'cap_com', 'stat_com', 'NOM', 'NOM_M', 'STATUT', 'POPULATION',
       'INSEE_DEP', 'INSEE_REG', 'geometry', 'surface'],
      dtype='object')


In [200]:
columns_to_drop = ['INSEE_COM', 'NOM', 'NOM_M', 'STATUT', 'POPULATION', 'INSEE_DEP', 'INSEE_REG']

df_velo = df_velo.drop(columns=columns_to_drop)


In [201]:
missing_values = df_velo.isna().sum()

print(missing_values)

nom_com                0
insee_com              0
longueur               0
longueur_pond_hw       0
longueur_pond_adg      0
difference_hw          0
difference_adg         0
cap_com              923
stat_com             923
geometry             923
surface              923
dtype: int64


In [202]:
df_velo = pd.merge(df_velo, communes, on='insee_com', how='left')

In [203]:
df_velo['geometry'] = df_velo['geometry_y']
df_velo['surface'] = df_velo['surface_y']
df_velo['population'] = df_velo['POPULATION']

In [204]:
columns_to_drop = ['POPULATION', 'id', 'ID','INSEE_CAN', 'INSEE_ARR', 'INSEE_COM', 'NOM', 'NOM_M', 'STATUT', 'INSEE_DEP', 'INSEE_REG', 'SIREN_EPCI', 'geometry_y', 'geometry_x', 'surface_x', 'surface_y']

df_velo = df_velo.drop(columns=columns_to_drop)


In [205]:
print(df_velo.shape)

(1005, 13)


Point d'étape : nous enregistrons cette base intermédiaire des vélos pour ne pas avoir à refaire toutes les étapes toujours.

In [206]:
# Sauvegarder le DataFrame au format GeoJSON

gdf_velo = gpd.GeoDataFrame(df_velo, geometry='geometry')

# Sauvegarder le GeoDataFrame au format GeoJSON
gdf_velo.to_file('df_velo.geojson', driver='GeoJSON')

## Données socio-démographiques sur l'Ile de France

In [207]:
# Charger dataframe df_velo

df_velo = gpd.read_file('df_velo.geojson')

In [208]:
df_velo.head(10)

,nom_com,insee_com,longueur,longueur_pond_hw,longueur_pond_adg,difference_hw,difference_adg,cap_com,stat_com,source,surface,population,geometry
0,10e Arrondissement,75110,60281,55764.25,51170.00,-4516.75,-9111.00,1619.0,55.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,2.890030e+06,86472,"POLYGON ((2.36471 48.88437, 2.36527 48.88433, ..."
1,11e Arrondissement,75111,90116,84508.75,73843.00,-5607.25,-16273.00,2126.0,57.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,3.650811e+06,145208,"POLYGON ((2.37690 48.87205, 2.37701 48.87195, ..."
2,12e Arrondissement,75112,201748,201066.75,202496.25,-681.25,748.25,2656.0,67.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,1.638084e+07,139297,"POLYGON ((2.39007 48.82569, 2.38909 48.82635, ..."
3,13e Arrondissement,75113,137731,128770.50,119280.50,-8960.50,-18450.50,2256.0,68.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,7.138222e+06,180005,"POLYGON ((2.34390 48.81577, 2.34394 48.81601, ..."
4,14e Arrondissement,75114,102618,94183.00,87486.75,-8435.00,-15131.25,1830.0,55.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,5.613081e+06,135592,"POLYGON ((2.34209 48.83834, 2.34208 48.83830, ..."
5,15e Arrondissement,75115,148909,136007.00,128724.50,-12902.00,-20184.50,3345.0,90.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,8.472060e+06,230981,"POLYGON ((2.26279 48.83392, 2.26356 48.83464, ..."
6,16e Arrondissement,75116,228240,226273.25,231651.00,-1966.75,3411.00,2129.0,65.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,1.640971e+07,165523,"POLYGON ((2.27995 48.87857, 2.28010 48.87852, ..."
7,17e Arrondissement,75117,114273,108548.00,100289.50,-5725.00,-13983.50,2085.0,63.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,5.665304e+06,166543,"POLYGON ((2.32742 48.88351, 2.32734 48.88349, ..."
8,18e Arrondissement,75118,118171,113091.00,98709.50,-5080.00,-19461.50,1798.0,61.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,5.995586e+06,192468,"POLYGON ((2.37030 48.90185, 2.37025 48.90177, ..."
9,19e Arrondissement,75119,133478,125675.25,128372.00,-7802.75,-5106.00,1762.0,61.0,IGN:EXPRESS-COG-CARTO-TERRITOIRE,6.784861e+06,184573,"POLYGON ((2.37690 48.87205, 2.37537 48.87343, ..."


## Données complémentaires : accidents de la route, transports écologiques alternatifs (réseau ferré, bus)

## Base finale avec la longueur, les indicateurs géographiques, et les variables socio-démographiques

In [ ]:
base = communes[['insee_com', 'POPULATION', 'surface', 'geometry']].merge(y, how='outer', on='insee_com')

In [ ]:
base.groupby('insee_com').sum(numeric_only = True).sort_values('longueur', ascending = False)

In [ ]:
base = gpd.GeoDataFrame(base, geometry='geometry')

# Analyse descriptive et spatiale

## Répartition spatiale des aménagements cyclables : piste cyclable et station vélib

In [ ]:
# Répartition des stations vélib

fig,ax = plt.subplots(figsize=(10, 10))
velib.plot(ax = ax, color = 'green')
com.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Emplacement des pistes cycables (point)

fig,ax = plt.subplots(figsize=(10, 10))
a_velo.plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
communes.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Répartition des pistes cyclabes en île de France

fig, ax = plt.subplots(figsize=(10, 10))
dissolved = base.dissolve(by='insee_com', aggfunc='sum').reset_index()
dissolved.plot(ax=ax, column="longueur", legend=True)
ax.set_axis_off()
legend = ax.get_legend()
plt.show()

### Statistiques descriptives sur les pistes cyclables

In [ ]:
df_amenagements.describe()

In [ ]:
sns.scatterplot(df_amenagements['longueur'])
plt.title('Distribution des pistes cyclables dans les communes')